In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
root_path='gdrive/My Drive/PLANT DISEASE DETECTION/'
# Proj Drive Folder
# use print(root_path) / !ls proj_folder_path  to check

In [3]:
# from google.colab import files
# files.upload()

In [4]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
# !kaggle datasets download -d koheidozono/newplantvillage -p '/content/gdrive/My Drive/PLANT DISEASE DETECTION/'

In [6]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [7]:
from zipfile import ZipFile
with ZipFile('/content/gdrive/MyDrive/PLANT DISEASE DETECTION/newplantvillage.zip', 'r') as zipObj:
  zipObj.extractall()
  print('Done')

Done


In [8]:
classes_train = sorted(list(filter(lambda x: os.path.isdir('/content/newplantvillage/' + x), os.listdir('/content/newplantvillage'))))

In [9]:
print(classes_train) #to show names of all classes

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Healthy_leaf', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


In [10]:
!pip install split_folders

In [11]:
import splitfolders
input_folder='/content/newplantvillage'
splitfolders.ratio(input_folder, output="output", seed=1337, ratio=(.8,.2)) #default values"

Copying files: 63987 files [00:45, 1417.89 files/s]


In [12]:
batch_size=128
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1/255)
train_generator=train_datagen.flow_from_directory(
    '/content/output/train',
    target_size=(200, 200),
    batch_size=batch_size,
    classes = classes_train,
    class_mode='categorical'
)
val_datagen=ImageDataGenerator(rescale=1/255)
val_generator=val_datagen.flow_from_directory(
    '/content/output/val',
    target_size=(200, 200),
    batch_size=batch_size,
    classes = classes_train,
    class_mode='categorical'
)

Found 51174 images belonging to 35 classes.
Found 12813 images belonging to 35 classes.


In [15]:
# Number of images per class in the training set
train_class_counts = train_generator.classes
train_class_counts_dict = {class_label: (train_class_counts == idx).sum()
                           for class_label, idx in train_generator.class_indices.items()}
print("Training set image counts per class:")
print(train_class_counts_dict)

# Number of images per class in the validation set
val_class_counts = val_generator.classes
val_class_counts_dict = {class_label: (val_class_counts == idx).sum()
                         for class_label, idx in val_generator.class_indices.items()}
print("Validation set image counts per class:")
print(val_class_counts_dict)


Training set image counts per class:
{'Apple___Apple_scab': 1612, 'Apple___Black_rot': 1589, 'Apple___Cedar_apple_rust': 1408, 'Apple___healthy': 1606, 'Blueberry___healthy': 1452, 'Cherry_(including_sour)___Powdery_mildew': 1346, 'Cherry_(including_sour)___healthy': 1460, 'Grape___Black_rot': 1510, 'Grape___Esca_(Black_Measles)': 1536, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 1377, 'Grape___healthy': 1353, 'Healthy_leaf': 806, 'Orange___Haunglongbing_(Citrus_greening)': 1608, 'Peach___Bacterial_spot': 1470, 'Peach___healthy': 1382, 'Pepper,_bell___Bacterial_spot': 1530, 'Pepper,_bell___healthy': 1590, 'Potato___Early_blight': 1551, 'Potato___Late_blight': 1551, 'Potato___healthy': 1459, 'Raspberry___healthy': 1424, 'Soybean___healthy': 1617, 'Squash___Powdery_mildew': 1388, 'Strawberry___Leaf_scorch': 1419, 'Strawberry___healthy': 1459, 'Tomato___Bacterial_spot': 1361, 'Tomato___Early_blight': 1536, 'Tomato___Late_blight': 1480, 'Tomato___Leaf_Mold': 1505, 'Tomato___Septoria_leaf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to display one image per class
def display_one_image_per_class(generator, class_indices):
    # Reverse the class indices dictionary to map index to class
    index_to_class = {v: k for k, v in class_indices.items()}

    # Dictionary to store one image per class
    images_per_class = {class_name: None for class_name in class_indices}

    # Iterate over the generator to find one image per class
    for _ in range(len(generator)):
        images, labels = next(generator)

        for i in range(images.shape[0]):
            class_index = np.argmax(labels[i])
            class_name = index_to_class[class_index]

            if images_per_class[class_name] is None:
                images_per_class[class_name] = images[i]

        # Break the loop if we have found an image for each class
        if all(img is not None for img in images_per_class.values()):
            break

    # Prepare a figure to plot the images
    plt.figure(figsize=(20, 25))

    for idx, (class_name, img) in enumerate(images_per_class.items()):
        plt.subplot(1, len(class_indices), idx + 1)
        plt.imshow(img)
        # plt.title(class_name)
        plt.axis('off')

    plt.show()

# Display one image per class in the training set
display_one_image_per_class(train_generator, train_generator.class_indices)


In [13]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # The first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a dense layer
    tf.keras.layers.Flatten(),
    # 128 neuron in the fully-connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    # 35 output neurons for 35 classes with the softmax activation
    tf.keras.layers.Dense(35, activation='softmax')
])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 16)      448       
                                                                 
 batch_normalization (Batch  (None, 198, 198, 16)      64        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 32)        4640      
                                                                 
 batch_normalization_1 (Bat  (None, 97, 97, 32)        128       
 chNormalization)                                                
                                                        

In [22]:
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy'])

epochs=12
steps_per_epoch=train_generator.n//train_generator.batch_size
validation_steps=val_generator.n//val_generator.batch_size

checkpoint = keras.callbacks.ModelCheckpoint("/content/abc.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

history=model.fit(
    x=train_generator,
    y=None,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=[checkpoint]
)

Epoch 1/12
399/399 [==============================] - ETA: 0s - loss: 0.8641 - accuracy: 0.7392
Epoch 1: val_loss improved from inf to 5.99998, saving model to /content/abc.h5
399/399 [==============================] - 332s 819ms/step - loss: 0.8641 - accuracy: 0.7392 - val_loss: 6.0000 - val_accuracy: 0.1587


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/12
399/399 [==============================] - ETA: 0s - loss: 0.2625 - accuracy: 0.9155
Epoch 2: val_loss improved from 5.99998 to 0.42928, saving model to /content/abc.h5
399/399 [==============================] - 263s 660ms/step - loss: 0.2625 - accuracy: 0.9155 - val_loss: 0.4293 - val_accuracy: 0.8606
Epoch 3/12
399/399 [==============================] - ETA: 0s - loss: 0.1469 - accuracy: 0.9514
Epoch 3: val_loss did not improve from 0.42928
399/399 [==============================] - 315s 789ms/step - loss: 0.1469 - accuracy: 0.9514 - val_loss: 0.5706 - val_accuracy: 0.8366
Epoch 4/12
399/399 [==============================] - ETA: 0s - loss: 0.0994 - accuracy: 0.9662
Epoch 4: val_loss improved from 0.42928 to 0.19909, saving model to /content/abc.h5
399/399 [==============================] - 301s 754ms/step - loss: 0.0994 - accuracy: 0.9662 - val_loss: 0.1991 - val_accuracy: 0.9383
Epoch 5/12
399/399 [==============================] - ETA: 0s - loss: 0.0745 - accuracy: 0.9

In [26]:
from tensorflow import keras
keras.models.save_model(model, "model.h5")

<ipython-input-26-02912c4d4e96>:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  keras.models.save_model(model, "model.h5")


In [27]:
model = tf.keras.models.load_model('/content/abc.h5')
model.evaluate(val_generator)

101/101 [==============================] - 47s 464ms/step - loss: 0.1993 - accuracy: 0.9382


[0.1993241161108017, 0.9381877779960632]

In [28]:

test_images = [f for f in os.listdir('/content/output/val')]
dataset = np.ndarray(shape=(len(test_images), 200, 200, 3))

import cv2

for i, file_name in enumerate(test_images):
  try:
    img=cv2.imread('/content/output/val' + file_name)
    img_resize=cv2.resize(img,(200,200))
    dataset[i]=np.array(img_resize)/255.0
  except:
    print(file_name)
dataset.shape

Strawberry___Leaf_scorch
Tomato___Septoria_leaf_spot
Pepper,_bell___Bacterial_spot
Apple___Black_rot
Tomato___Early_blight
Tomato___Target_Spot
Grape___Esca_(Black_Measles)
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Tomato___healthy
Grape___healthy
Pepper,_bell___healthy
Cherry_(including_sour)___Powdery_mildew
Peach___Bacterial_spot
Apple___Apple_scab
Cherry_(including_sour)___healthy
Tomato___Tomato_mosaic_virus
Peach___healthy
Potato___Late_blight
Potato___healthy
Soybean___healthy
Tomato___Late_blight
Strawberry___healthy
Squash___Powdery_mildew
Potato___Early_blight
Tomato___Spider_mites Two-spotted_spider_mite
Blueberry___healthy
Orange___Haunglongbing_(Citrus_greening)
Tomato___Bacterial_spot
Healthy_leaf
Grape___Black_rot
Tomato___Leaf_Mold
Raspberry___healthy
Apple___Cedar_apple_rust
Apple___healthy
Tomato___Tomato_Yellow_Leaf_Curl_Virus


(35, 200, 200, 3)

In [31]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/model.h5')
## For TFLite model
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()
open('model.tflite', 'wb').write(tflite_model)
## For TFLite optimized model
converter.optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_opt_model=converter.convert()
open('model_opt.tflite', 'wb').write(tflite_opt_model)

249024

In [32]:
from google.colab import files
files.download('model.h5')  ##generated model file
files.download('model.tflite') ##generated model tflite converted file
files.download('model_opt.tflite') ##generated model tflite converted file

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
class_names = [
    "Pepper,_bell___Bacterial_spot",
    "Apple___Black_rot",
    "Peach___healthy",
    "Strawberry___Leaf_scorch",
    "Grape___healthy",
    "Apple___Cedar_apple_rust",
    "Orange___Haunglongbing_(Citrus_greening)",
    "Blueberry___healthy",
    "Grape___Black_rot",
    "Potato___Early_blight",
    "Cherry_(including_sour)___Powdery_mildew",
    "Cherry_(including_sour)___healthy",
    "Grape___Esca_(Black_Measles)",
    "Potato___healthy",
    "Healthy_leaf",
    "Tomato___Leaf_Mold",
    "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    "Tomato___Tomato_mosaic_virus",
    "Tomato___healthy",
    "Apple___Apple_scab",
    "Tomato___Target_Spot",
    "Tomato___Septoria_leaf_spot",
    "Peach___Bacterial_spot",
    "Tomato___Spider_mites Two-spotted_spider_mite",
    "Tomato___Bacterial_spot",
    "Strawberry___healthy",
    "Tomato___Early_blight",
    "Tomato___Late_blight",
    "Apple___healthy",
    "Pepper,_bell___healthy",
    "Raspberry___healthy",
    "Squash___Powdery_mildew",
    "Soybean___healthy",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    "Potato___Late_blight"
]


In [35]:
import cv2
import numpy as np

def load_and_preprocess_image(file_path):
    # Load image using OpenCV
    img = cv2.imread(file_path)
    # Resize image to match model input shape (200x200 pixels)
    img_resized = cv2.resize(img, (200, 200))
    # Convert pixel values to float and normalize (assuming RGB image)
    img_processed = img_resized.astype(np.float32) / 255.0
    # Expand dimensions to match the input shape expected by the model
    img_processed = np.expand_dims(img_processed, axis=0)
    return img_processed

# Example usage for loading a single image
image_path = '/content/output/val/Grape___Esca_(Black_Measles)/0075b632-2e34-4e4f-9697-fe2b332b7ef8___FAM_B.Msls 4399.JPG'
test_image = load_and_preprocess_image(image_path)


In [36]:
# Predict probabilities for the image
predictions = model.predict(test_image)

# Decode the predictions (if needed)
predicted_class = np.argmax(predictions, axis=-1)

print("Predicted class:", predicted_class)
print("Predicted probabilities:", predictions)


1/1 [==============================] - 1s 1s/step
Predicted class: [26]
Predicted probabilities: [[2.0354126e-11 5.6343719e-10 2.9204395e-07 8.3364982e-11 5.9292637e-13
  1.2715497e-10 5.1424307e-17 8.0752559e-04 3.9367063e-04 1.7712640e-14
  4.7571738e-15 3.4492245e-18 4.3027357e-22 7.7792856e-13 1.3583039e-13
  8.2791517e-11 1.2204480e-06 1.1430353e-06 2.2685511e-14 4.0000123e-06
  1.6394799e-14 2.2607449e-11 1.1475616e-19 1.9065196e-13 4.8401750e-20
  1.0359667e-15 9.9878329e-01 4.8760613e-09 3.6257991e-06 7.0817741e-08
  1.2491417e-09 5.1764296e-06 1.0600128e-14 1.0795280e-08 1.0613495e-11]]


In [37]:
class_names[16]

'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)'

In [ ]:
import time

while True:
    print("Keeping the kernel alive...")
    time.sleep(600)  # Sleep for 5 minutes (300 seconds)

Keeping the kernel alive...
